In [6]:
#Read data
import cvxpy as cvx
import numpy as np
import math
import mosek
# to run the following commands, please follow  https://www.instructables.com/id/Call-MATLAB-Script-and-Function-From-Python/
import matlab.engine
eng = matlab.engine.start_matlab()

case ="case141"
ret = eng.Read_RadialNet_Matpower(case)  #case33bw, case123  # these command is equivalent to run Read_Matpower_Radial(case33bw) in Matlab
import scipy.io as spio #Reading Matlab mat data as Python dictionary
data = spio.loadmat('Radial_Netdata', squeeze_me=True) #taken from Matlab command Read_Matpower_Radial(Matpower_case_format);
#data contains the following parameters
#'Sbase', 'Pgmax', 'Pgmin', 'Qgmax', 'Qgmin', 'GenVarCost0', 'GenVarCost', 'G', 'B', 'MVAlim', 'connex', 'Pd', 'Qd', 'Vmax', 'Vmin','Gen2Bus'
r = data['r']
x = data['x']
slack_bus =0
#slack_bus =55 #case 123 slackbus is 113 (Python index count from 0)
#slack_bus =113
I = len(data['Pd'])
G = data['G']
B = -data['B']
MVAlim =data['MVAlim']
status = data['status']
def connex(i,j,data): #return 1 of (i,j) or (j,i) is a "connected" line
    temp = False
    for line in range(data['L']):
        z1= (i== data['from_bus'][line]-1) and (j== data['to_bus'][line]-1)
        z2= (j== data['from_bus'][line]-1) and (i== data['to_bus'][line]-1)
        z3= (data['status'][line]==1) #some lines are open switches
        z= (z1 or z2) and z3
        temp = temp or z
    return temp

In [3]:
print(MVAlim)
if case =="case18ieee":
    scale =10
else:
    scale =1    
data['Sbase']= scale*data['Sbase']
r = r*scale
x = x*scale
data['gs'] = data['gs']/scale
data['bs'] = data['bs']/scale

[100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100
 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100
 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100
 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100
 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100
 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100
 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100
 100 100 100 100 100 100 100 100 100 100 100 100 100 100]


In [5]:
from math import sin, cos, tan, pi
rat_coeff = spio.loadmat('rationalcoff', squeeze_me=True)

print(rat_coeff)

def linearSOCP(x1,x2,x3,xi,eta,approximation_para):
    #x1^2+x2^2 <= x3^2
    v= approximation_para
    const=[]
    const += [xi[v] <= x3 ] #(a[n]+b[n]) ]
    const += [eta[v] <= xi[v]*tan(pi/(2**(v+1)))]
    for k in range(1,v+1):
        const += [xi[k]== xi[k-1]*cos(pi/(2**(k+1))) + eta[k-1]*sin(pi/(2**(k+1)))]
        const += [eta[k]>= -xi[k-1]*sin(pi/(2**(k+1))) + eta[k-1]*cos(pi/(2**(k+1)))]
        const += [eta[k]>= xi[k-1]*sin(pi/(2**(k+1))) - eta[k-1]*cos(pi/(2**(k+1)))]              
    const += [xi[0] >=  x1 ]
    const += [eta[0] >= x2 ] 
    const += [eta[0] >= -x2 ]
    return const

print(rat_coeff['rationalcoff'][0])

def rotatedCONE(x1,x2,x3,x4):
    #x1^2 + x2^2 <= x3*x4
    # this is equivalent to x1^2 +x2^2 + 1/4*(x3-x4)^2 <= 1/4*(x3+x4)^2
    return  [cvx.norm2(cvx.bmat( [[x1, x2, 0.5*(x3-x4)]])) <= 0.5*(x3+x4)] 

def rotatedCONE2(x1,x2,x3):
    #x1^2  <= x3*x4
    # this is equivalent to x1^2 +1/4*(x2-x3)^2 <= 1/4*(x2+x3)^2
    return  [cvx.norm2(cvx.bmat( [[x1, 0.5*(x2-x3)]])) <= 0.5*(x2+x3)] 

def Rational_linearSOCP(x1,x2,x3,xi,eta,rat_coeff):
    #x1^2+x2^2 <= x3^2
    a_coff = rat_coeff['rationalcoff'][0]
    b_coff = rat_coeff['rationalcoff'][1]
    c_coff = rat_coeff['rationalcoff'][2]
    v= 12
    const=[]
    const += [xi[v] <= x3 ] #(a[n]+b[n]) ]
    const += [b_coff[v]*eta[v] <= xi[v]*a_coff[v]]
    for k in range(1,v+1):
        const += [c_coff[k]*xi[k]== xi[k-1]*b_coff[k] + eta[k-1]*a_coff[k]]
        const += [c_coff[k]*eta[k]>= -xi[k-1]*a_coff[k] + eta[k-1]*b_coff[k]]
        const += [c_coff[k]*eta[k]>= xi[k-1]*a_coff[k] - eta[k-1]*b_coff[k]]              
    const += [xi[0] >=  x1 ]
    const += [eta[0] >= x2 ] 
    const += [eta[0] >= -x2 ]
    return const

{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Fri May 14 11:18:38 2021', '__version__': '1.0', '__globals__': [], 'rationalcoff': array([[      0,     105,      36,       9,      36,      35,     140,
            280,     560,     559,    2236,    4472,    4471],
       [      0,      88,      77,      40,     323,     612,    4899,
          19599,   78399,  156240, 1249923, 4999695, 9994920],
       [      0,     137,      85,      41,     325,     613,    4901,
          19601,   78401,  156241, 1249925, 4999697, 9994921]])}
[   0  105   36    9   36   35  140  280  560  559 2236 4472 4471]


In [ ]:
# Distribution power flows of Rabih A. Jabr
P = cvx.Variable((I,I))
Q = cvx.Variable((I,I))
R = cvx.Variable(data['L'],nonneg=True) 
T = cvx.Variable(data['L'])            
v     = cvx.Variable(I) 
Isq  = cvx.Variable(data['L'],nonneg=True)

bus_Pinjection = [-sum(P[i,j]  for j in range(I) if connex(i,j,data)) for i in range(I)]
bus_Qinjection = [-sum(Q[i,j]  for j in range(I) if connex(i,j,data)) for i in range(I)]

Pgrid = -bus_Pinjection[slack_bus]*data['Sbase']
Qgrid = -bus_Qinjection[slack_bus]*data['Sbase']
    
const = []
const += [v[i] >= (data['Vmin'][i])**2 for i in range(I) if i != slack_bus]
const += [v[i] <= (data['Vmax'][i])**2 for i in range(I) if i != slack_bus]
const += [v[slack_bus] == 1]
 
dlmp_bus_index1=len(const)   
const += [(bus_Pinjection[i]+data['gs'][i]*v[i])== data['Pd'][i]/data['Sbase']  for i in range(I) if i != slack_bus]
dlmp_bus_index2= len(const)   

const += [(bus_Qinjection[i]+data['bs'][i]*v[i]) == data['Qd'][i]/data['Sbase'] for i in range(I) if i != slack_bus] 

for line in range(data['L']):
    i = data['from_bus'][line]-1
    j = data['to_bus'][line]-1       
    if status[line] ==1:
        const += [P[i,j] == (r[line]*v[i] - r[line]*R[line]+ x[line]*T[line])/(r[line]**2 + x[line]**2)]
        const += [Q[i,j] == (x[line]*v[i] - x[line]*R[line]- r[line]*T[line])/(r[line]**2 + x[line]**2)]
        const += [P[j,i] == (r[line]*v[j] - r[line]*R[line]- x[line]*T[line])/(r[line]**2 + x[line]**2) ]
        const += [Q[j,i] == (x[line]*v[j] - x[line]*R[line]+ r[line]*T[line])/(r[line]**2 + x[line]**2) ]        
        const += [Isq[line]  == (v[i]+v[j] - 2*R[line])/(r[line]**2 + x[line]**2) ]
        #repalce MVAlim
#        const += [(MVAlim[line]/data['Sbase']) >= cvx.norm2(cvx.bmat([[P[i,j], Q[i,j]]]))   ]
        # by constraint on current limits
#        const += [Isq[line] <= 4**2   ]
#        const += [(MVAlim[line]/data['Sbase'])**2 >= cvx.square(P[i,j]) + cvx.square(Q[i,j])]
    
index1 = len(const)

approximation_para =12
xi = cvx.Variable( (data['L'],approximation_para+1),nonneg=True)
eta =cvx.Variable( (data['L'],approximation_para+1),nonneg=True)  
xi1 = cvx.Variable((data['L'],approximation_para+1),nonneg=True)
eta1 =cvx.Variable((data['L'],approximation_para+1),nonneg=True) 
RL  = cvx.Variable(data['L'], nonneg=True )                 
                  
for line in range(data['L']):
    if status[line] ==1:
        i = data['from_bus'][line]-1
        j = data['to_bus'][line]-1
#        const += [cvx.quad_over_lin(cvx.bmat([[R[line], T[line]]]), v[i]) <= v[j]]
#        const += [cvx.norm2(cvx.bmat([[ R[line], T[line], 0.5*(v[i] - v[j])  ]] ))  <=  0.5*(v[i]+ v[j]) ]
#        const += [cvx.norm2(cvx.bmat([[ R[line], T[line], 0.5*(v[i] - v[j])  ]] ))  <=  0.5*(v[i]+ v[j]) ]
        const += Rational_linearSOCP(RL[line], 0.5*(v[i] - v[j]), 0.5*(v[i]+ v[j]), xi1[line], eta1[line],rat_coeff)
        const += Rational_linearSOCP(R[line], T[line], RL[line], xi[line], eta[line],rat_coeff)
index2 = len(const)
          
prob = cvx.Problem(cvx.Minimize(Pgrid), const) 

prob.solve(solver='CPLEX', verbose=True, warm_start = True) 
"""
prob.solve(solver = 'MOSEK', 
                    mosek_params = {#mosek.dparam.optimizer_max_time:  100.0,
                                    mosek.iparam.intpnt_solve_form:   mosek.solveform.dual},
                    warm_start = True,
                    verbose = True)
"""

print('loss:', Pgrid.value-sum(data['Pd']), 'MW')
#print('objective'obj)
#print('voltages:', [math.sqrt(v[i].value) for i in range(I)])
print('== Active Power Flow == (in pu)')
#print('We now check if the Second order conic constraints are binding:')
#print('SOCP dual:', [const[i].dual_value for i in range(index1,index2)])
#print ('If all dual variables are positive, the relaxation is excat')
print(" BUS DATA")
print('imported active power from main grid:', Pgrid.value,'MW')
print('imported reactive power from main grid:', Qgrid.value,'MVAr')
print("Bus    Voltage")
for i in range(I):
    print(i+1, '  ', round(math.sqrt(v[i].value), 5))
print('================================================================================')    
for line in range(data['L']):   
    if status[line] ==1:
        print(math.sqrt(Isq.value[line]))
print('=================================END CVXPY==============================')
print('================================================================================')
#print(MATPOWER_RESULT)

In [ ]:
#Branch flow model of Farivar and S. Low
const =[]
#data = spio.loadmat('Radial_Netdata', squeeze_me=True)
r = data['r']
x = data['x']
P = cvx.Variable((I,I))
Q = cvx.Variable((I,I))
Vsq  = cvx.Variable(I)
Isq  = cvx.Variable(shape=(data['L'],), nonneg=True)

"""
P_outflow_of = [0 for i in range(I)]
Q_outflow_of = [0 for i in range(I)]
P_inflow_of  = [data['gs'][i]*Vsq[i] for i in range(I)]
Q_inflow_of  = [data['bs'][i]*Vsq[i] for i in range(I)]

for j in range(I):
    for k in range(I):
        for line in range(data['L']):
            if status[line]==1:
                if (j==data['from_bus'][line]-1) and (k==data['to_bus'][line]-1):
                    P_outflow_of[j] +=  P[j,k]
                    Q_outflow_of[j] +=  Q[j,k]   
for j in range(I):
    for i in range(I):
        for line in range(data['L']):
            if status[line]==1:
                if (i==data['from_bus'][line]-1) and (j==data['to_bus'][line]-1):
                    P_inflow_of[j] +=  (P[i,j] - r[line]*Isq[line])
                    Q_inflow_of[j] +=  (Q[i,j] - x[line]*Isq[line])                       
"""
P_outflow_of = [ sum(P[j, data['to_bus'][line]-1] for line in range(data['L']) 
                  if  ((status[line]==1) and (j==data['from_bus'][line]-1)))  for j in range(I)]     

Q_outflow_of = [ sum(Q[j, data['to_bus'][line]-1] for line in range(data['L']) 
                  if  ((status[line]==1) and (j==data['from_bus'][line]-1)) ) for j in range(I)] 

P_inflow_of  = [  sum(P[data['from_bus'][line]-1,j] - r[line]*Isq[line] 
                        for line in range(data['L']) if  ((status[line]==1) and (j==data['to_bus'][line]-1)) ) 
                        for j in range(I)]  

Q_inflow_of  = [  sum(Q[data['from_bus'][line]-1,j] - x[line]*Isq[line] 
                        for line in range(data['L'])  if  ((status[line]==1) and (j==data['to_bus'][line]-1)))  
                        for j in range(I)] 

index1 = len(const)
for line in  range(data['L']): 
    if status[line]==1:
        const +=rotatedCONE(P[data['from_bus'][line]-1,data['to_bus'][line]-1], Q[data['from_bus'][line]-1,data['to_bus'][line]-1], Vsq[data['from_bus'][line]-1],Isq[line] )
index2 = len(const)

for line in range(data['L']):
    if status[line] ==1:
        i = data['from_bus'][line]-1
        j = data['to_bus'][line]-1
        const +=[ Vsq[j] == Vsq[i] - 2*(r[line]*P[i,j]+x[line]*Q[i,j])+ (r[line]**2 + x[line]**2)*Isq[line] ]
#       replace aprperant line flows
#        const += [(MVAlim[line]/data['Sbase']) >= cvx.norm2(cvx.bmat( [[ P[i,j], Q[i,j] ]])) ]  
#       by line current limits (4pu) as Lucient Lobo
#        const += [Isq[line] <= 4**2 ] 
const +=[ Vsq[i] >= (data['Vmin'][i])**2 for i in range(I) if i!=slack_bus]
const +=[ Vsq[i] <= (data['Vmax'][i])**2 for i in range(I) if i!=slack_bus]
const +=[ Vsq[slack_bus] == 1]


dlmp_bus_index1 = len(const)
const += [data['gs'][j]*Vsq[j]  -(P_outflow_of[j] - P_inflow_of[j]) == data['Pd'][j]/data['Sbase'] for j in range(I) if j !=slack_bus]
dlmp_bus_index2 = len(const)

const += [data['bs'][j]*Vsq[j] -(Q_outflow_of[j] - Q_inflow_of[j]) == data['Qd'][j]/data['Sbase'] for j in range(I) if j !=slack_bus]

Pgrid = P_outflow_of[slack_bus]*data['Sbase']
Qgrid = Q_outflow_of[slack_bus]*data['Sbase']  
  
prob = cvx.Problem(cvx.Minimize(Pgrid), const) 

#prob = Problem(Minimize(obj*data['Sbase']), const) 
prob.solve(solver='MOSEK', verbose=True, warm_start = True) # method ='dccp', max_iter = 10, tau=100, ccp_times=1, mu=0.5 ) # method ='dccp', max_iter = 10, solver = 'SCS' )
 
print('imported grid:', Pgrid.value, 'MW')
print('imported reactive power from main grid:', Qgrid.value,'MVAr')

print('loss:', Pgrid.value-sum(data['Pd']))   

print('voltage:', [math.sqrt(Vsq[i].value) for i in range(I)])
"""
print('== Active Power Flow == (in pu)')

for i in range(I):
    for j in range(I):
        if connex[i,j]==1:
            print('P', 'line', i, '-->', j, ': ', P.value[i,j])

print('== Reactive Power Flow == (in pu)')
      
for i in range(I):
    for j in range(I):
        if connex[i,j]==1:
            print('Q', 'line', i, '-->', j, ': ', Q.value[i,j])      
"""            
print ('We now check if the Second order conic constraints are binding:')
print('SOCP dual:', [const[i].dual_value for i in range(index1,index2)])
print ('If all dual variables are positive, the relaxation is excat')

print('We now show DLMP:')
print('DLMP:', [-const[i].dual_value for i in range(dlmp_bus_index1, dlmp_bus_index2)])

#Check with Rah model
print(" BUS DATA")
print('imported active power from main grid:', Pgrid.value,'MW')
print('imported reactive power from main grid:', Qgrid.value,'MVAr')
print("Bus    Voltage")
for i in range(I):
    print(i+1, '  ', round(math.sqrt(Vsq[i].value), 5))


In [ ]:
#Branch flow model of Farivar and S. Low (Cleaner version)
const =[]
data = spio.loadmat('Radial_Netdata', squeeze_me=True)
r = data['r']
x = data['x']
P = cvx.Variable((I,I))
Q = cvx.Variable((I,I))
S = cvx.Variable(shape=(data['L'],), nonneg=True)
Vsq  = cvx.Variable(I)
Isq  = cvx.Variable(shape=(data['L'],), nonneg=True)

P_outflow_of = [0 for i in range(I)]
Q_outflow_of = [0 for i in range(I)]
P_inflow_of  = [data['gs'][i]*Vsq[i] for i in range(I)]
Q_inflow_of  = [data['bs'][i]*Vsq[i] for i in range(I)]

for j in range(I):
    for k in range(I):
        for line in range(data['L']):
            if status[line]==1:
                if (j==data['from_bus'][line]-1) and (k==data['to_bus'][line]-1):
                    P_outflow_of[j] +=  P[j,k]
                    Q_outflow_of[j] +=  Q[j,k]                   
for j in range(I):
    for i in range(I):
        for line in range(data['L']):
            if status[line]==1:
                if (i==data['from_bus'][line]-1) and (j==data['to_bus'][line]-1):
                    P_inflow_of[j] +=  (P[i,j] - r[line]*Isq[line])
                    Q_inflow_of[j] +=  (Q[i,j] - x[line]*Isq[line])                       

index1 = len(const)
for line in  range(data['L']): 
    if status[line]==1:
#        const +=[ cvx.quad_over_lin(cvx.bmat(F[line]),  Vsq[data['from_bus'][line]-1]  ) <= Isq[line] ]  
        const +=[ cvx.norm2(cvx.bmat( [[S[line], 0.5*(Vsq[data['from_bus'][line]-1]-Isq[line])]] ))
                                <= 0.5*(Vsq[data['from_bus'][line]-1]+Isq[line]) ]  

index2 = len(const)
const += [MVAlim[line]/data['Sbase'] >= S[line]]
for line in range(data['L']):
    if status[line] ==1:
        i = data['from_bus'][line]-1
        j = data['to_bus'][line]-1
        const +=[ Vsq[j] == Vsq[i] - 2*(r[line]*P[i,j]+x[line]*Q[i,j])+ (r[line]**2 + x[line]**2)*Isq[line] ]
        const += [S[line] >= cvx.norm2(cvx.bmat( [[ P[i,j], Q[i,j] ]])) ]  
#
const +=[ Vsq[i] >= (data['Vmin'][i])**2 for i in range(I) if i!=slack_bus]
const +=[ Vsq[i] <= (data['Vmax'][i])**2 for i in range(I) if i!=slack_bus]
const +=[ Vsq[slack_bus] == 1]

dlmp_bus_index1 = len(const)
const += [ -(P_outflow_of[j] - P_inflow_of[j]) == data['Pd'][j]/data['Sbase'] for j in range(I) if j !=slack_bus]
dlmp_bus_index2 = len(const)

const += [ -(Q_outflow_of[j] - Q_inflow_of[j]) == data['Qd'][j]/data['Sbase'] for j in range(I) if j !=slack_bus]

Pgrid = P_outflow_of[slack_bus]*data['Sbase']
Qgrid = Q_outflow_of[slack_bus]*data['Sbase']  

prob = cvx.Problem(cvx.Minimize(Pgrid), const) 

#prob = Problem(Minimize(obj*data['Sbase']), const) 
#prob.solve( solver='SCS',  warm_start = True, max_iters= 100000000 ) #verbose=True,
prob.solve( solver='MOSEK',  warm_start = True) #verbose=True,

# method ='dccp', max_iter = 10, tau=100, ccp_times=1, mu=0.5 ) # method ='dccp', max_iter = 10, solver = 'SCS' )
 
print('imported grid:', Pgrid.value, 'MW')
print('imported reactive power from main grid:', Qgrid.value,'MVAr')

print('loss:', Pgrid.value-sum(data['Pd']))   

print('voltage:', [math.sqrt(Vsq[i].value) for i in range(I)])

"""
print('== Active Power Flow == (in pu)')
for i in range(I):
    for j in range(I):
        if connex[i,j]==1:
            print('P', 'line', i, '-->', j, ': ', P.value[i,j])

print('== Reactive Power Flow == (in pu)')     
for i in range(I):
    for j in range(I):
        if connex[i,j]==1:
            print('Q', 'line', i, '-->', j, ': ', Q.value[i,j])      
"""  

print ('We now check if the Second order conic constraints are binding:')
print('SOCP dual:', [const[i].dual_value for i in range(index1,index2)])
print ('If all dual variables are positive, the relaxation is excat')

print('We now show DLMP:')
print('DLMP:', [-const[i].dual_value for i in range(dlmp_bus_index1, dlmp_bus_index2)])

#Check with Rah model
print(" BUS DATA")
print('imported active power from main grid:', Pgrid.value,'MW')
print('imported reactive power from main grid:', Qgrid.value,'MVAr')
print("Bus    Voltage")
for i in range(I):
    print(i+1, '  ', round(math.sqrt(Vsq[i].value), 5))